# **Two-Sample t-Test**

A **two-sample t-test** checks whether the means of two independent samples are significantly different from each other.

---

## **1. Hypotheses**

**Null Hypothesis (H₀):**  
Mean of group 1 = Mean of group 2  
$$
H_0: \mu_1 = \mu_2
$$

**Alternative Hypothesis (H₁):**  
Means are not equal  
$$
H_1: \mu_1 \neq \mu_2
$$

---

## **2. Test Statistic**

### **(a) Equal Variances Assumed (Pooled t-test)**

$$
t = \dfrac{\bar{X}_1 - \bar{X}_2}{S_p \sqrt{\dfrac{1}{n_1} + \dfrac{1}{n_2}}}
$$

where  

$$
S_p = \sqrt{\dfrac{(n_1 - 1)S_1^2 + (n_2 - 1)S_2^2}{n_1 + n_2 - 2}}
$$

---

### **(b) Unequal Variances Assumed (Welch’s t-test)**

$$
t = \dfrac{\bar{X}_1 - \bar{X}_2}{\sqrt{\dfrac{S_1^2}{n_1} + \dfrac{S_2^2}{n_2}}}
$$

Degrees of freedom are approximated by:

$$
df = \dfrac{\left(\dfrac{S_1^2}{n_1} + \dfrac{S_2^2}{n_2}\right)^2}
{\dfrac{\left(\dfrac{S_1^2}{n_1}\right)^2}{n_1 - 1} + \dfrac{\left(\dfrac{S_2^2}{n_2}\right)^2}{n_2 - 1}}
$$

---

## **3. Degrees of Freedom (Equal Variance Case)**

$$
df = n_1 + n_2 - 2
$$

---

## **4. Decision Rule**

If  
$$
|t_{calculated}| > t_{critical}
$$  
→ **Reject H₀** (significant difference between means)

Otherwise → **Fail to reject H₀** (no significant difference)

---


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import stats, t

In [2]:
data = pd.read_csv('weights_equal.csv')

In [3]:
male_pop = data[data['Gender'] =='Male']
female_pop = data[data['Gender'] =='Female']

In [4]:
female_pop

,Gender,Weight
100,Female,64.197035
101,Female,66.078733
102,Female,75.231055
103,Female,74.272592
104,Female,69.853689
...,...,...
195,Female,66.715770
196,Female,58.008058
197,Female,79.477107
198,Female,69.198221


In [5]:
sample_m = male_pop['Weight'].sample(30)
sample_f = female_pop['Weight'].sample(30)

In [6]:
sample_f.mean()

70.2152640214999

In [7]:
sample_m.mean()

70.93338169432997

In [40]:
def T_Pooled_test(sample_1, sample_2, tails):
    neumerator = sample_1.mean() - sample_2.mean()
    df = ( (len(sample_1) + len(sample_2)) - 2)
    
    
    se = (((len(sample_1) - 1) * sample_1.var()) + ((len(sample_2) - 1) * sample_2.var()))
    se = se / ((len(sample_1) + len(sample_2)) - 2)
    se = np.sqrt(se)
    
    
    denumerator = se * np.sqrt(((1/len(sample_1)) + 1/len(sample_2)))
    T_value = neumerator / denumerator
    return T_value, p_value(T_value, tails, df)
    
    
    
def p_value(t_value, tails, df):
    if tails == 2:
        p_value = tails * (1 - t.cdf(abs(t_value), df))
    else:
        p_value =  t.cdf(t_value, df)
    return p_value
    

In [41]:
print(T_Pooled_test(sample_m, sample_f, 1))
print("Right side area", 1-(T_Pooled_test(sample_m, sample_f, 1)[1]))
print("Left side area", (T_Pooled_test(sample_m, sample_f, 1)[1]))

(0.3876720287954892, 0.6501610726280675)
Right side area 0.34983892737193245
Left side area 0.6501610726280675


In [10]:
print("Two tail test",T_Pooled_test(sample_m, sample_f, 2))

Two tail test (0.3876720287954892, 0.6996778547438649)


## **Welch’s t-Test (Unequal Variance Case)**

Now, we perform a **two-sample t-test** assuming **unequal variances**.

This is called **Welch’s t-test**.  
It is used when the variances of the two samples are not equal.

The test statistic is:

$$
t = \frac{\bar{X}_1 - \bar{X}_2}{
\sqrt{\frac{S_1^2}{n_1} + \frac{S_2^2}{n_2}}
}
$$

and the degrees of freedom are estimated using the **Welch–Satterthwaite equation**:

$$
df = 
\frac{
\left(\frac{S_1^2}{n_1} + \frac{S_2^2}{n_2}\right)^2
}{
\frac{\left(\frac{S_1^2}{n_1}\right)^2}{n_1 - 1} +
\frac{\left(\frac{S_2^2}{n_2}\right)^2}{n_2 - 1}
}
$$


In [45]:
def T_welchs_test(sample_1, sample_2, tails):
    neumerator = (sample_1.mean() - sample_2.mean())
    denumerator = np.sqrt(((sample_1.var() / (len(sample_1) - 1)) + (sample_2.var() / (len(sample_2) - 1))))
    T_value = neumerator / denumerator
    return T_value, P_value(T_value, degree_of_freedom(sample_1, sample_2), tails), degree_of_freedom(sample_1, sample_2)
    
def degree_of_freedom(sample_1, sample_2):
    neumerator = ((sample_1.var() / len(sample_1)) + (sample_2.var() / len(sample_2))) ** 2
    denumerator = ((((sample_1.var() / len(sample_1)) ** 2) / (len(sample_1) - 1)) + (((sample_2.var() / len(sample_2)) ** 2) / (len(sample_2) - 1)))
    df = neumerator / denumerator
    return df
def P_value(T_value, df, tails):
    if tails == 2:
        P_value = tails * (1 - t.cdf(abs(T_value), df))
    else:
        P_value = t.cdf(T_value, df)
    return P_value
print(T_welchs_test(sample_m, sample_f, 1))
print("Right side area", 1-(T_welchs_test(sample_m, sample_f, 1)[1]))
print("Left side area", (T_welchs_test(sample_m, sample_f, 1)[1]))

(0.3811560684462428, 0.6476466638789968, 49.864230625800445)
Right side area 0.35235333612100317
Left side area 0.6476466638789968


In [43]:
print(T_Pooled_test(sample_m, sample_f, 1))

(0.3876720287954892, 0.6501610726280675)


In [32]:
stats.ttest_ind(sample_m, sample_f, equal_var=False)

C:\Users\bittu\AppData\Local\Temp\ipykernel_13832\3161199544.py:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  stats.ttest_ind(sample_m, sample_f, equal_var=False)


TtestResult(statistic=0.3876720287954892, pvalue=0.6999087110789854, df=49.864230625800445)

In [33]:
stats.ttest_ind(sample_m, sample_f, equal_var=True)

C:\Users\bittu\AppData\Local\Temp\ipykernel_13832\480581880.py:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  stats.ttest_ind(sample_m, sample_f, equal_var=True)


TtestResult(statistic=0.3876720287954891, pvalue=0.6996778547438651, df=58.0)